You are in a dimly lit game room. There is an old couch and a locked door to the north.
> look around
You see a couch and a locked door.
> examine couch
You find a small key tucked in the cushions.
> take key
You take the key.
> use key
You unlock the door.
> open door
Congratulations! You escape the haunted mansion.


In [15]:
rooms = {
    'main room': { # room 1. Player starts here. 
        'description': "~The Main Room is dimly lit. The floor creaks. Cobwebs hang from the walls.\n" 
        "You can see a dusty old couch and a antique grand piano.~", # Player wakes up on couch and sees room with piano
        'connections': ['bedroom'], # Player must access Bedroom 1 to continue
        'items': ["couch", "grand piano"] 
    },
    'bedroom': { # room 2, connects with main room and living room
        'description': 'The Bedroom walpaper is yellow and peeling. with a large, soft bed.', # player must search bed to find master key for exit
        'connections': ["main room", 'dining room'], 
        'items': ["bed", 'silver key']
    },
    'dining room': { # room 3, player can search dining table and/or use key to exit house
        'description': 'An elegant dining room with an grand dining table.',
        'connections': ["bedroom", 'outside'],
        'items': ["dining table"]
    },
    "outside": { # final challenge. game ends when player exits house
        "description": "You are outside. Fresh air at last!",
        "connections": ["dining room"],
    },
    # Additional rooms...
}


In [16]:
class Game:
    def __init__(self, room, item):
        self.location = room
        self.item = item
        self.room_examined = False
        self.has_key = False
        self.inventory = []  # New line
        
    def select_item(self, new_item):
        # this function sets an atribute to a new value
        self.item = new_item

    def examine_room(self):
        self.room_examined = True
        room_dict = rooms[self.location]
        return room_dict
    

    def move_rooms(self, new_location):
        self.location = new_location
        self.room_examined = False
        self.item = None
    
    def print_room(self):
        room_dict = rooms[self.location]
        return room_dict["description"]

    def take_key(self):
        self.has_key = True
    
    def take_item(self, item):  # New method
        self.inventory.append(item)
        print(f"You took the {item}")
    


In [11]:
player = Game("main room", "couch")

In [ ]:
# display string function
# function has to split string into chars, capitalise the 1st char, then join the string
def display_string(dis):


In [ ]:
# Game loop
game_running = True
while game_running:
    print(f"You are in the {player.location}. What do you want to do?", flush = True)
    
    # Show available connections
    current_room = rooms[player.location]
    connections = current_room['connections']
    
    if player.room_examined == True:
        print(f'''
            Press 1 to examine the {player.location}
            Press 2 to move to another room
            Press 3 to examine items
            Press 4 to select an item
            Type "exit" to exit game  
            ''', flush = True)
    else:
        print(f'''
            Press 1 to examine the {player.location}
            Press 2 to move to another room
            Press 3 to examine items
            Type "exit" to exit game  
            ''', flush = True)   
    
 
    
    user_input = input("What do you want to do? ")
    
    if user_input == "1":
        room_description = player.print_room()
        print(room_description)
        if len(connections) > 0: # turn into a function, 
            print("Rooms you can move to:")
            for i, room in enumerate(connections):
                print(f"    - {room}")
    elif user_input == "2":
        if len(connections) > 0:
            new_room = input(f"Which room? {connections}: ").lower()
            if new_room in connections:
                # Check if trying to go outside
                if new_room == "outside" and player.has_key:
                    print("You use the Silver Key to unlock the door and escape!")
                    print("Congratulations! You won!")
                    game_running = False
                elif new_room == "outside" and not player.has_key:
                    print("The door is locked. You need a key to escape.")
                else:
                    player.move_rooms(new_room)
                    print(f"You moved to {new_room}") # print(you have moved to {display_string(new_room})
            else:
                print("You can't go there from here.")
        else:
            print("No exits available.")
    elif user_input == "3":
        room_info = player.examine_room()
        print(room_info["items"])
        print("Do you want to select an item?")
    elif user_input == "4":
        print("Which items do you want to select? Type item name:")
        user_input = input("Which items do you want to select? Type item name: ")
        if user_input.lower() == "silver key":
            player.take_key()
            print("Congratulations! This key is special. Use it to escape!")
        else: 
            print("Nothing special about this item. Keep searching!")    
    elif user_input.lower() == "exit":
        print("You've ended the game. Good-bye!")
        game_running = False
    else:
        print("I don't understand that command.")

You are in the main room. What do you want to do?

            Press 1 to examine the main room
            Press 2 to move to another room
            Press 3 to examine items
            Press 4 to select an item
            Type "exit" to exit game  
            
